In [1]:
import pandas as pd
import numpy as np

import re

import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime
from difflib import get_close_matches

from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import GridSearchCV


from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import svm

from sklearn.metrics import f1_score

from xgboost import XGBClassifier, XGBRegressor, plot_importance

from sklearn.utils import shuffle

In [2]:
class DecisionMaking:
    """
    A class that takes pre-trained models and product the next predicted values
    
    list_to_add_unit_hour - list of rolling averages to add in the unit of hour, e.g.
                                list_to_add_unit_hour = [1, 2, 4] , will produce RAs of
                                1H, 2H, 4H (window calculated by dividing by the index_unit)

    index_unit - unit of index (whatever was used in the grouper), e.g. '15min' (takes minutes)


    """
    
    def __init__(self, columns_to_norm, base, list_to_add_unit_hour, index_unit, model_avg, model_high, model_low):
        
        self.columns_to_norm = columns_to_norm
        self.base = base
        self.list_to_add_unit_hour = list_to_add_unit_hour
        self.index_unit = index_unit
        
        self.model_avg  = model_avg
        self.model_high = model_high
        self.model_low  = model_low
        
        assert base not in columns_to_norm, "base should not be in the list of columns, as it will be dividing by 0"
    
    def get_raw_data(self, df):
        """
        raw 1 minute df
        """
        assert df.index.dtype == '<M8[ns]', "Index should be time"
        self.raw_1min_df = df.sort_index(ascending=True).copy(deep=True)
        
        return None
    
    def window_groupby(self, freq, agg_grouping, return_results=False):
        
        """
        return_results to see the table returned, otherwise, save at: self.df_windowed
        """
        
        df=self.raw_1min_df.copy(deep=True)
        grouper_key = self.raw_1min_df.index.name
        assert grouper_key != None, "no grouper key"
        
        df_temp = df.reset_index().groupby(pd.Grouper(key=grouper_key, freq=freq)).agg(agg_grouping)
        if 'open_time' in agg_grouping.keys():
            df_temp.columns = [open_time_refine(col) for col in df_temp.columns]

        df_temp['avg'] = df_temp['quote_asset_volume']/df_temp['volume']
        df_temp['avg'] = df_temp['avg'].fillna(method='ffill')  ## when volume is 0
        
        if return_results:
            return df_temp
            
        else:
            self.df_windowed = df_temp.copy(deep=True)
            return None


    
    def add_rolling_averges(self, return_results=False):
        """
        INPUT:
        return_results - boolean, defult False
        
        OUTPUT:
        (if return_results = True), dataframe with added rolling averages
        (else, save at self.df_wind_ra)
        """

        df_temp = self.df_windowed.copy(deep=True)

        rows = float(self.index_unit.replace('min',''))/60

        for ra in self.list_to_add_unit_hour:

            df_temp['ra_{x}H'.format(x=ra)] = df_temp['avg'].rolling(window=int(ra/rows)).mean()

        if return_results:
            print(".dropna(axis=0) not applied here")
            return df_temp
            
        else:
            self.df_wind_ra = df_temp.dropna(axis=0).copy(deep=True)
            return None


    
    def normalising_data(self, return_results=False):
        """
        INPUT:
        return_results - boolean, defult False

        OUTPUT:
        (if return_results = True), normalised dataframe
        (else, save at self.df_normalised_input)
        """
        
        df_temp = self.df_wind_ra.copy(deep=True)

        for col in self.columns_to_norm:
            df_temp[col] = 100 * ((df_temp[col].astype(np.float)/df_temp[self.base]) - 1)

        if return_results:
            return df_temp
            
        else:
            self.df_normalised_input = df_temp.copy(deep=True)
            return None
        
    def _return_predicted_vales(self, pred, base_value):
        final_value = (1.0 + pred/100) * base_value
        return final_value
    
    def get_predictions(self, ml_low, ml_avg, ml_high):
        """
        INPUTS:
        ml_low = machine learning model for predicting low
        ml_avg = machine learning model for predicting average
        ml_high = machine learning model for predicting high
        
        Return
        predicted: average, low, and high from the models
        """
        y_pred_avg_decision = ml_avg.predict(self.df_normalised_input[self.columns_to_norm].values)
        y_pred_low_decision = ml_low.predict(self.df_normalised_input[self.columns_to_norm].values)
        y_pred_high_decision = ml_high.predict(self.df_normalised_input[self.columns_to_norm].values)

        y_pred_unscaled_avg = self._return_predicted_vales(y_pred_a    vg_decision[0], self.df_normalised_input[self.base].values[0])
        y_pred_unscaled_low = self._return_predicted_vales(y_pred_low_decision[0], self.df_normalised_input[self.base].values[0])
        y_pred_unscaled_high = self._return_predicted_vales(y_pred_high_decision[0], self.df_normalised_input[self.base].values[0])
            
        return {'pred_avg':y_pred_unscaled_avg,
                'pred_low':y_pred_unscaled_low,
                'pred_high':y_pred_unscaled_high}

In [ ]:
class MrMarket:
    def __init__(self, df):
        self.df = df
    
    def _tick_tock(self, time_now):
        !!! - 1minute
        self.time_return = time_now - 1min
    
    def call(self, time_now=):
        """returns the data for the last minute"""
        
#         time_now = '2017-09-12 06:10:00'
        self._tick_tock(time_now)
    
        return df.loc[self.time_return]
    
class Broker(MrMarket):
    def __init__(self, df, usd_wallet, coin_wallet, maker_fee=0.1, taker_fee=0.2):
        MrMarket.__init__(self, df)
        self.usd_wallet = usd_wallet
        self.coin_wallet = coin_wallet
        
        ## maybe add floating position here? self.usd_float, self.crypto_float 
        ## what happens if trading multiple crypto? (they are all from the same wallet)
        
        self.maker_fee = maker_fee
        self.taker_fee = taker_fee
 
        
        self.next_action = 'open_buy_position'  ## can be from {'open_buy_position', 'open_sell_position',
                                                ##              'close_buy_position', 'close_sell_position'}
        
        self.open_order = False # True means currently there is a buy order; to buy crypto using USD
        self.current_low = None  ## will be compared to lows
        self.current_high = None  ## will be compared to highs
        self.pred_low = None
        self.pred_high = None
    
        self.trade_log = [{"time_of_event" : datetime.now(),  ## if back testing, now() shoud be the max time 
                          "event_name"    : "initialisation",
                          "usd_wallet"    : self.usd_wallet, 
                          "coin_wallet"   : self.coin_wallet,
                          "next_action"   : self.next_action,
                           "open_order"   : self.open_order,
                           "pred_low"     : self.pred_low,
                           "pred_high"    : self.pred_high,
                           "true_low"     : self.current_low,
                           "true_high"    : self.current_high
                         }]

    def get_log(self):
        return self.trade_log
    
    def _add_to_log(self, time, event_name, usd_wallet, coin_wallet, next_action):
        self.trade_log.append(
                         {"time_of_event" : time,  ## if back testing, now() shoud be the max time 
                          "event_name"    : event_name,
                          "usd_wallet"    : usd_wallet, 
                          "coin_wallet"   : coin_wallet,
                          "next_action"   : next_action,
                          "open_order"    : self.open_order,
                          "pred_low"     : self.pred_low,
                          "pred_high"    : self.pred_high,
                          "true_low"     : self.current_low,
                          "true_high"    : self.current_high
        })
        
        return None

    def _open_buy_position(self, pred_low):
        self.buy_position = pred_low  ## open a position on the low price, can make adjustments here
        self.open_order = True
        self.next_action = "close_buy_position"
        
        self._add_to_log(self.time,
                         "open_buy_position",
                         self.usd_wallet,
                         self.coin_wallet,
                         self.next_action
                        )
        
        return None 
    
    def _execute_buy_position(self, true_low):
        if true_low < self.buy_position:
            
            self.open_order = False
            self.coin_wallet = self.coin_wallet + float(self.usd_wallet / self.buy_position) * (100 - self.taker_fee)/100
            self.usd_wallet = self.usd_wallet - self.usd_wallet ## (second one should be actual usd spent)
            self.next_action = "open_sell_position"
            
            self._add_to_log(self.time, 
                             "close_buy_position",
                             self.usd_wallet,
                             self.coin_wallet,
                             self.next_action
                            )
            return None
            
        else:
            self._add_to_log(self.time,
                             "waiting_to_close_buy_position",
                             self.usd_wallet,
                             self.coin_wallet,
                             self.next_action
                            )
            return None
            
    
    def _open_sell_position(self, pred_high):
        self.sell_position = pred_high
        self.open_order = True
        self.next_action = "close_sell_position"   
        
        self._add_to_log(self.time, 
                         "open_sell_position",
                         self.usd_wallet,
                         self.coin_wallet,
                         self.next_action
                        )
        
        return None
    
    
    def _execute_sell_position(self, true_high):
        if true_high > self.sell_position:
            self.open_order = False
            
            self.usd_wallet = self.usd_wallet + self.sell_position * self.coin_wallet * (100 - self.taker_fee)/100
            self.coin_wallet = self.coin_wallet - self.coin_wallet

            self.next_action = "open_buy_position"
            self._add_to_log(self.time,
                             "close_sell_position",
                             self.usd_wallet,
                             self.coin_wallet,
                             self.next_action
                            )
            return None
    
        else:
            self._add_to_log(self.time, 
                             "waiting_to_close_sell_position",
                             self.usd_wallet,
                             self.coin_wallet,
                             self.next_action
                            )
            return None
    
    def cancel_position(self):
        self.open_order = False
        if 
        self.next_action = 'open_buy_position'

        self._add_to_log(self.time, 
                             "cancel_trade",
                             self.usd_wallet,
                             self.coin_wallet,
                             self.next_action
                            )
        
        return None
    

In [ ]:
class TradingBot:
    def __init__(self):
        pass
    
    def get_market_value(self, current_low, current_high, time):
        
    